In [1]:
import pandas as pd

In [9]:
tr = pd.read_csv('data/train.csv', index_col='Id')
sub = pd.read_csv('data/sample-submission.csv')

In [12]:
tr['Prediction'].describe()

count     250000
unique         2
top            b
freq      164333
Name: Prediction, dtype: object